In [1]:
import torch

In [2]:
import logging
from hydra import compose, initialize
from omegaconf import OmegaConf

from autoeye.config import Config

if __name__ == "__main__":
    # global initialization
    initialize(version_base=None, config_path="conf", job_name="test_app")
    cfg = compose(config_name="config")
    print(OmegaConf.to_yaml(cfg))
    log = logging.getLogger(__name__)
    Config.setup(cfg, log, train=True)

data:
  csv_files:
  - ./data/train.csv
  - ./data/test.csv
model:
  name: autoeye
  backbone: resnet
  version: 1.0
optim:
  beta1: 0.9
  beta2: 0.95
  weight_decay: 0.1
hyper:
  pretrained: true
  batch_size: 8
  dropout: 0.2
  lr: 0.03
  epochs: 10
  eval_iters: 1
  use_amp: true



In [3]:
from autoeye.model import AutoEye


model = AutoEye()
checkpoint = torch.load("./models/autoeye_resnet.pt")
model.load(checkpoint)

/home/spectre/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-2])
resnet

Using cache found in /home/spectre/.cache/torch/hub/pytorch_vision_v0.10.0
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [14]:
model.backbone = resnet

In [8]:
checkpoint = {
    "backbone": model.backbone.state_dict(),
    "classifier": model.classifier.state_dict(),
    "optimizer": checkpoint["optimizer"],
    "scaler": checkpoint["scaler"],
    "epochs": 0,
}

In [10]:
torch.save(checkpoint, "./models/autoeye_resnet.pt")